In [1]:
import os
# Set the working directory to /workspace
os.chdir("/workspace")
# Verify
print("Current Directory:", os.getcwd())  # Output: /workspace

Current Directory: /workspace


In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl

import pandas as pd
import numpy as np
import seaborn as sns
from termcolor import colored
import category_encoders as ce

import plotly.express as px  
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from ipywidgets import interactive, IntSlider, Checkbox, FloatSlider, BoundedIntText, Text, Layout, HBox, VBox, Output, Textarea
from IPython.display import display, clear_output, HTML
import warnings
import sys
from scipy.stats import spearmanr
from ydata_profiling import ProfileReport
import math

warnings.filterwarnings('ignore')

In [3]:
# Please define the value of 'bad' in the target column:
# Please define the value of 'bad' in the target column:
# Please define the value of 'bad' in the target column:
# Please define the value of 'bad' in the target column:

bad = 0
# bad = 1

In [4]:
# df_all = pd.read_csv('UCI_Credit_Card_v2.csv') 
# feat = n = 'PAY_AMT6'
# target_col = 'label'

# ###########################################

df_all = pd.read_csv('df3.csv') 

# Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
# feat = n = 'revol_util'
feat = n = 'total_rev_hi_lim'
target_col = 'good_bad'
# total_pymnt
# int_rate

In [5]:
# CapprossBins the auto binning


# ##########################################
# ########## Ensure float else display the issue in the feature
df = df_all[[feat, target_col]].copy()
df['original'] = df[feat].astype(str)

# Coerce to numeric
df[feat] = pd.to_numeric(df['original'], errors='coerce')
df['is_non_numeric'] = df[feat].isna()

# Check for non-numeric values
if df['is_non_numeric'].any():
    print(f"⚠️ Non-numeric values detected in column: {feat}")

    non_numeric_counts = df[df['is_non_numeric']]['original'].value_counts()

    if not non_numeric_counts.empty:
        plt.figure(figsize=(8, 5))
        non_numeric_counts.plot(kind='bar', color='orange')
        plt.title(f'Non-Numeric Values in {feat}')
        plt.xlabel('Original Value')
        plt.ylabel('Count')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    # Stop execution
    sys.exit(f"❌ Stopping execution due to non-numeric values in '{feat}'")
else:
    pass


# ###########################################
# ########## Adjusts the target column based on the 'bad' class indicator.

positive_label = 1
negative_label = 0
positive = 'good'    # reference to positive_label
negative = 'bad'   # reference to negative_label


def good_has_ones(df, target_col, bad=bad):
    """
    Adjusts the target column in a binary classification DataFrame based on the 'bad' class indicator.

    """
    positive_count = len(df[df[target_col] == 1])
    negative_count = len(df[df[target_col] == 0])

    if (positive_count > negative_count) and (bad == 0):
        data = df.copy()
        print('1s are greater than 0s, where bad has 0')
    elif (positive_count > negative_count) and (bad == 1):
        data = df.drop(columns=[target_col])
        data[target_col] = 1 - df[target_col]
        print('1s are greater than 0s, where bad has 1')
    elif (negative_count > positive_count) and (bad == 0):
        data = df.copy()
        print('0s are greater than 1s, where bad has 0')
    else:
        data = df.drop(columns=[target_col])
        data[target_col] = 1 - df[target_col]
        print('0s are greater than 1s, where bad has 1')
        print('Target is inverted')
    
    return data



# ##########

new_df = good_has_ones(df, target_col, bad=bad)
filtered_df = pd.DataFrame()
# majority or good has target value = 1
# minority or bad has target value = 0


# ##########################################################################
# ########### OUTLIERS

# # Calculate Q1 and Q3
Q1 = new_df[feat].quantile(0.25)
Q3 = new_df[feat].quantile(0.75)
Q4 = new_df[feat].quantile(0.9)

# Compute IQR
IQR = Q3 - Q1

# Define bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# ##############################Filter the DataFrame
filtered_df = new_df[(new_df[feat] >= lower_bound) & (new_df[feat] <= upper_bound)]
# filtered_df = new_df[(new_df[feat] >= Q1) & (new_df[feat] <= Q4)]


# print()
# print(Q1)
# print(Q3)
# print(Q4)
# print()
# print(filtered_df[feat].min())
# print(filtered_df[feat].max())


# ####################################################################################################
# ####################################################################################################


# Columns rename
for x in [new_df, filtered_df]:
    x.rename(columns={feat: 'feature', target_col: 'target'}, inplace=True)


# ###########################################################################################
# CapprossBins the auto binning

# Helper Functions
def filter_special_values(data_input, special_val=None):
    """Separates special values from regular data for bin calculations."""
    data = data_input.copy()
    
    if special_val is not None:
        if np.isnan(special_val):
            # Handle NaN special values
            special_mask = data['feature'].isna()
        else:
            # Handle numeric special values
            special_mask = data['feature'] == special_val
        
        special_data = data[special_mask].copy()
        regular_data = data[~special_mask].copy()
        
        return regular_data, special_data
    else:
        return data, pd.DataFrame()
    

def calculate_metrics(data_input, bins, special_data=None):
    """Calculates extended binning metrics with full dataset totals."""
    data = data_input.copy()

    # (we expect data['bin'] was just created via pd.cut AND include_lowest=True)
    if 'bin' not in data.columns:
            raise ValueError("Data must have a 'bin' column assigned before calling calculate_metrics.")
    
    
    # --- BEGIN: preserve special rows, then possibly collapse an empty first bin 
    # 1. Pull out any special rows so we don’t overwrite them
    is_special = data['bin'] == 'special'
    special_rows = data[is_special].copy()
    regular_rows = data[~is_special].copy()

    ivals = pd.IntervalIndex.from_breaks(bins, closed='left')
    counts = (
        pd.cut(regular_rows['feature'],
                bins=ivals,
            include_lowest=True,
            right=False)
        .value_counts()
        .reindex(ivals, fill_value=0)
    )
    # Check if -ve value is a boundary before collapsing
    has_special_boundary = 0 in bins

# ############################################################
# The below was change to ensure that if -99,999 is not mentioned as special value, then if exists it will be taken as normal number
# ##########################################################
    # # If the first interval is empty, drop its boundary (index 1) to merge
    # if counts.iloc[0] == 0 and len(bins) > 2:
    #     bins_to_use = np.delete(bins, 1)
    # else:
    #     bins_to_use = bins

    bins_to_use = bins
# ##########################################################

    # Now bin ONLY the regular rows with bins_to_use
    regular_rows['bin'] = pd.cut(
        regular_rows['feature'],
        bins=bins_to_use,
        include_lowest=True,
        right=False
    )

    # 2. Stitch special rows back (if any)
    if not special_rows.empty:
        special_rows['bin'] = 'special'
        data = pd.concat([regular_rows, special_rows], ignore_index=True)
    else:
        data = regular_rows
    # --- END: preserve & collapse ---
    
    
    # Extract special values if they exist
    has_special = 'special' in data['bin'].values
    regular_data = data[data['bin'] != 'special'] if has_special else data
    special_data = data[data['bin'] == 'special'] if has_special else pd.DataFrame()

    # YSR:; find min of regular to exclude the special value
    regular_feature_values = regular_data['feature']
    range_min = regular_feature_values.min()
    range_max = regular_feature_values.max()

    grouped = data.groupby('bin', observed=False)
    bin_total = grouped.size()

    if bin_total.sum() == 0:
        print("Warning: No data points fall into the generated bins.")
        empty_series = pd.Series(dtype=float)
        return (empty_series,)*16

    # Calculate metrics for all bins (including special)
    good = grouped['target'].sum()
    bad = bin_total - good
    
    # Calculate rates and proportions
    bad_rate = (bad / bin_total).fillna(0)
    good_rate = (good / bin_total).fillna(0)


    # --- BEGIN: Calculate bin ranges (widths) ---

    bin_ranges = []
    for bin_val in bin_total.index:
        if bin_val == 'special':
            bin_ranges.append('special')
            continue

        # ###################################################################################################################
        # The below was change to ensure that if -99,999 is not mentioned as special value, then if exists it will be taken as normal number
        # #####################################################################################################################################
        # # Extract left/right bounds

        # left_bound = bin_val.left
        # right_bound = bin_val.right

        # # Use actual bin boundaries, don't replace with data min/max
        # if np.isinf(left_bound) and left_bound < 0:
        #     effective_left = range_min
        # else:
        #     effective_left = left_bound

        # if np.isinf(right_bound) and right_bound > 0:
        #     effective_right = range_max
        # else:
        #     effective_right = right_bound

        # width_value = int(abs(effective_right - effective_left))
        # bin_ranges.append(f"{width_value:,}")   # add , for thousands

        
        left_bound = bin_val.left
        right_bound = bin_val.right

        # Use actual bin boundaries, don't replace with data min/max
        if np.isinf(left_bound) and left_bound < 0:
            effective_left = left_bound  # Keep -inf
            left_str = "-inf"
        else:
            effective_left = left_bound
            left_str = f"{effective_left:,.0f}"

        if np.isinf(right_bound) and right_bound > 0:
            effective_right = right_bound  # Keep +inf  
            right_str = "inf"
        else:
            effective_right = right_bound
            right_str = f"{effective_right:,.0f}"

        # Only calculate width for finite boundaries
        if np.isfinite(effective_left) and np.isfinite(effective_right):
            width_value = int(abs(effective_right - effective_left))
            bin_ranges.append(f"{width_value:,}")
        else:
            # For infinite boundaries, show the range instead of width
            bin_ranges.append(f"[{left_str}, {right_str})")
        # ########################################################################

    bin_range = pd.Series(bin_ranges, index=bin_total.index)



    
    # --- END: Calculate bin ranges (widths) ---

    
    # Use FULL dataset totals including special values
    total_good = data['target'].sum()
    total_bad = len(data) - total_good
    
    p_good = good / total_good if total_good > 0 else 0
    p_bad = bad / total_bad if total_bad > 0 else 0
    
    # Accumulated percentages
    accum_p_good = p_good.cumsum()
    accum_p_bad = p_bad.cumsum()

    epsilon = 1e-6
    woe = np.log((p_good + epsilon) / (p_bad + epsilon))
    iv_components = (p_good - p_bad) * woe
    iv_per_bin = iv_components
    iv_total = iv_components.sum()  # Includes all bins, including 'special', per standard practice

    p = p_good
    q = p_bad
    m = 0.5 * (p + q)
    kl_p_m = p * np.log((p + epsilon) / (m + epsilon))
    kl_q_m = q * np.log((q + epsilon) / (m + epsilon))
    js_per_bin = (0.5 * (kl_p_m + kl_q_m)).fillna(0)
    js_total = js_per_bin.sum()

    gini_per_bin = 2 * bad_rate * good_rate
    total_pop = bin_total.sum()
    gini_total = (gini_per_bin * (bin_total / total_pop)).sum() if total_pop > 0 else 0

    final_index = grouped.groups.keys()

    # ############################################################
    # ############################################################
    # ############################################################
    # ############################################################
    
    # final_index = grouped.groups.keys()

 

    # # 1. build the full list of intervals
    # bins_intervals = pd.IntervalIndex.from_breaks(bins, closed='left')
    # full_intervals = list(bins_intervals)

    # # 2. get counts for each (zero if empty)
    # counts = data.groupby('bin').size().reindex(full_intervals, fill_value=0)

    # # 3. if the very first interval is empty, merge it into the next
    # if counts.iloc[0] == 0 and len(full_intervals) > 1:
    #     # make a new interval from -inf to the right edge of the second
    #     right = full_intervals[1].right
    #     merged = pd.Interval(-np.inf, right, closed='left')
    #     # keep merged as first, then skip the old first and second
    #     plot_index = [merged] + full_intervals[2:]
    # else:
    #     plot_index = full_intervals

    # # 4. finally, if you have a 'special' bin, add it
    # if 'special' in data['bin'].values:
    #     plot_index.append('special')

    # final_index = plot_index
    # ############################################################
    # ############################################################
    # ############################################################
    # ############################################################

    
    return (
        bad_rate.reindex(final_index, fill_value=0),
        woe.reindex(final_index, fill_value=0),
        bin_total.reindex(final_index, fill_value=0),
        bin_range.reindex(final_index, fill_value=0),
        range_min,
        range_max,
        iv_per_bin.reindex(final_index, fill_value=0), 
        iv_total,
        gini_per_bin.reindex(final_index, fill_value=0), 
        gini_total,
        js_per_bin.reindex(final_index, fill_value=0), 
        js_total,
        p_bad.reindex(final_index, fill_value=0),
        good.reindex(final_index, fill_value=0),      # Good_count
        bad.reindex(final_index, fill_value=0),       # Bad_count
        p_good.reindex(final_index, fill_value=0),    # P_Good
        good_rate.reindex(final_index, fill_value=0), # Good Rate
        accum_p_good.reindex(final_index, fill_value=0),
        accum_p_bad.reindex(final_index, fill_value=0)
    )
# ##########################################################################
# ###########################################################################
def create_initial_bins_with_infinity(data, n_bins):
    """
    FIXED: Always creates bins spanning from -inf to +inf regardless of constraints.
    This ensures consistent bin range behavior.
    """
    if data.empty:
        raise ValueError("Cannot create bins from empty data")
    
    # Generate quantiles for internal boundaries only
    quantiles = np.linspace(0, 1, n_bins + 1)[1:-1]  # Exclude 0 and 1
    internal_boundaries = data['feature'].quantile(quantiles).unique()
    
    # Remove any infinite values from internal boundaries
    internal_boundaries = internal_boundaries[np.isfinite(internal_boundaries)]
    
    # Always create bins with -inf and +inf boundaries
    bins = np.concatenate(([-np.inf], internal_boundaries, [np.inf]))
    bins = np.unique(bins)  # Remove duplicates and sort
    
    return bins
# ##########################################################################
# ###########################################################################

def enforce_population_constraints(data_input, bins, min_pct=0.05, max_pct=0.20):
    """Population constraint enforcement - now works with consistent -inf/+inf bins"""
    data = data_input.copy()
    current_bins = np.unique(bins)
    max_iterations = 70
    iteration = 0

    while iteration < max_iterations:
        if len(current_bins) < 2: 
            break
            
        temp_data = data.copy()
        temp_data['bin'] = pd.cut(temp_data['feature'], bins=current_bins, include_lowest=True, duplicates='drop', right=False)
        bin_counts = temp_data['bin'].value_counts(sort=False, normalize=True)
        
        if bin_counts.empty:
            break
            
        min_pop_actual = bin_counts.min()
        max_pop_actual = bin_counts.max()

        if min_pop_actual >= min_pct and max_pop_actual <= max_pct: 
            break

        violation_indices = np.where((bin_counts < min_pct) | (bin_counts > max_pct))[0]
        if len(violation_indices) == 0: 
            break
            
        idx = violation_indices[0]

        if bin_counts.iloc[idx] < min_pct:
            # Merge bins by removing internal boundary
            if len(current_bins) > 2:
                # Find the internal boundary to remove (never remove -inf or +inf)
                delete_idx = idx + 1 if idx < len(bin_counts) - 1 else idx
                if 0 < delete_idx < len(current_bins) - 1:  # Don't remove first (-inf) or last (+inf)
                    current_bins = np.delete(current_bins, delete_idx)
                else: 
                    break
            else: 
                break
        else:
            # Split bins by adding internal boundary
            if idx + 1 < len(current_bins) - 1:  # Don't split at +inf boundary
                if pd.api.types.is_numeric_dtype(current_bins.dtype) and current_bins[idx] != current_bins[idx + 1]:
                    # Only add boundary between finite values
                    if np.isfinite(current_bins[idx]) and np.isfinite(current_bins[idx + 1]):
                        mid = (current_bins[idx] + current_bins[idx + 1]) / 2
                        if mid > current_bins[idx] and mid < current_bins[idx + 1]:
                            current_bins = np.insert(current_bins, idx + 1, mid)
                            current_bins = np.unique(current_bins)
                        else: 
                            break
                    else: 
                        break
                else: 
                    break
            else: 
                break
                
        iteration += 1
        if iteration == max_iterations:
            print("Warning: Max iterations reached in enforce_population_constraints.")
            
    return np.unique(current_bins)

def merge_bins_by_bad_rate(data_input, bins, threshold):
    """Bad rate merging - preserves -inf/+inf boundaries"""
    data = data_input.copy()
    current_bins = np.unique(bins)
    max_iterations = 70
    iteration = 0

    while iteration < max_iterations:
        if len(current_bins) <= 2: 
            break
            
        temp_data = data.copy()
        temp_data['bin'] = pd.cut(temp_data['feature'], bins=current_bins, include_lowest=True, duplicates='drop', right=False)
        grouped = temp_data.groupby('bin', observed=False)
        bin_totals = grouped.size()
        target_sum = grouped['target'].sum()
        bad_sum = bin_totals - target_sum
        bad_rates = (bad_sum / bin_totals).fillna(0).to_numpy()

        if len(bad_rates) <= 1: 
            break
            
        diffs = np.abs(np.diff(bad_rates))
        if len(diffs) == 0 or np.min(diffs) >= threshold: 
            break

        min_idx = np.argmin(diffs)
        boundary_to_remove = min_idx + 1
        
        # Only remove internal boundaries (not -inf or +inf)
        if 0 < boundary_to_remove < len(current_bins) - 1:
            current_bins = np.delete(current_bins, boundary_to_remove)
        else: 
            break
            
        iteration += 1
        if iteration == max_iterations:
            print("Warning: Max iterations reached in merge_bins_by_bad_rate.")
            
    return np.unique(current_bins)


def detect_monotonic_trend(bad_rates):
    if len(bad_rates) <= 1: return True
    diffs = np.diff(bad_rates)
    tolerance = 1e-9
    inc = np.sum(diffs > tolerance)
    dec = np.sum(diffs < -tolerance)
    if inc == 0 and dec == 0: return True
    return inc >= dec


def enforce_monotonic_bad_rate(data_input, bins):
    """Monotonic enforcement - preserves -inf/+inf boundaries"""
    data = data_input.copy()
    current_bins = np.unique(bins)
    max_iterations = 70
    iteration = 0

    while iteration < max_iterations:
        if len(current_bins) <= 2: 
            break
            
        temp_data = data.copy()
        temp_data['bin'] = pd.cut(temp_data['feature'], bins=current_bins, include_lowest=True, duplicates='drop', right=False)
        grouped = temp_data.groupby('bin', observed=False)
        bin_totals = grouped.size()
        target_sum = grouped['target'].sum()
        bad_sum = bin_totals - target_sum
        bad_rate = (bad_sum / bin_totals).fillna(0).to_numpy()

        if len(bad_rate) <= 1: 
            break
            
        trend_up = detect_monotonic_trend(bad_rate)
        diffs = np.diff(bad_rate)
        tolerance = 1e-9
        is_monotonic = False
        if trend_up and np.all(diffs >= -tolerance): 
            is_monotonic = True
        elif not trend_up and np.all(diffs <= tolerance): 
            is_monotonic = True
        if is_monotonic: 
            break

        violation_found_idx = -1
        for i, d in enumerate(diffs):
            if (trend_up and d < -tolerance) or (not trend_up and d > tolerance):
                violation_found_idx = i
                break
                
        if violation_found_idx == -1: 
            break
            
        boundary_to_remove = violation_found_idx + 1
        
        # Only remove internal boundaries (not -inf or +inf)
        if 0 < boundary_to_remove < len(current_bins) - 1:
            current_bins = np.delete(current_bins, boundary_to_remove)
        else: 
            break
            
        iteration += 1
        if iteration == max_iterations:
            print("Warning: Max iterations reached in enforce_monotonic_bad_rate.")
            
    return np.unique(current_bins)

def plot_binning(bad_rate, woe, bin_sizes, bin_range, range_min, range_max, iv_per_bin, iv_total, gini_per_bin, gini_total, js_per_bin, js_total, bins, p_bad, feature_name, output_area, good_count, bad_count, p_good, good_rate, accum_p_good, accum_p_bad, bin_purpose):

    with output_area:

        output_area.clear_output(wait=True)
        if bad_rate.empty or len(bins) < 2:
            print("Cannot generate plot/table: Insufficient data or bins.")
            return None

        common_index = bad_rate.index
        labels = []
        bin_counter = 1
        
        for bin_label in common_index:
            if isinstance(bin_label, str):
                labels.append(bin_label)  # For 'special' bin
            elif isinstance(bin_label, pd.Interval):
                # ==============================================================
                # MODIFIED LABEL GENERATION WITH DYNAMIC BRACKETS
                #
                # ==============================================================
                # Get interval properties
                closed = bin_label.closed
                left = bin_label.left
                right = bin_label.right
                
                # Determine brackets based on interval closure and infinity
                left_bracket = '[' if bin_label.closed_left else '('
                right_bracket = ']' if bin_label.closed_right else ')'
                
                # Force open brackets for infinity
                if np.isinf(left) and left < 0:
                    left_bracket = '('
                if np.isinf(right) and right > 0:
                    right_bracket = ')'
                
                # ###############Format numbers (2 decimal places for finite values)
                left_str = f"{left:.2f}" if not np.isinf(left) else "-inf"
                right_str = f"{right:.2f}" if not np.isinf(right) else "inf"
                # # ##############Format numbers (0 decimal places for finite values)
                # left_str = f"{left:.0f}" if not np.isinf(left) else "-inf"      # YSR: left_str = f"{left:.0f}" works!!
                # right_str = f"{right:.0f}" if not np.isinf(right) else "inf"    # YSR: right_str = f"{right:.0f}" works!!
         


                labels.append(f"Bin_{bin_counter}       <br>"
                              f"{left_bracket}{left_str}, {right_str}{right_bracket}"
                              )
                
                # labels.append(f"Bin_{bin_counter} <br>{left_bracket}{left_str}, {right_str}{right_bracket}")

                bin_counter += 1
                # ==============================================================
            else:
                labels.append(str(bin_label))

        # Reindex all series to ensure alignment
        woe = woe.reindex(common_index).fillna(0)
        bin_sizes = bin_sizes.reindex(common_index).fillna(0)
        bin_range = bin_range.reindex(common_index).fillna(0)
        iv_per_bin = iv_per_bin.reindex(common_index).fillna(0)
        gini_per_bin = gini_per_bin.reindex(common_index).fillna(0)
        js_per_bin = js_per_bin.reindex(common_index).fillna(0)
        p_bad = p_bad.reindex(common_index).fillna(0)
        good_count = good_count.reindex(common_index).fillna(0)
        bad_count = bad_count.reindex(common_index).fillna(0)
        p_good = p_good.reindex(common_index).fillna(0)
        good_rate = good_rate.reindex(common_index).fillna(0)
        accum_p_good = accum_p_good.reindex(common_index).fillna(0)
        accum_p_bad = accum_p_bad.reindex(common_index).fillna(0)

        total_pop = bin_sizes.sum()
        total_range = bin_range.sum()
        pop_pct = (bin_sizes / total_pop) if total_pop > 0 else pd.Series(np.zeros(len(bin_sizes)), index=bin_sizes.index)

        # Create comprehensive metrics table
        metrics_df = pd.DataFrame({
            'Bin': labels,
            'Range': bin_range.values,
            'Bad Rate': bad_rate.values,
            'Good Rate': good_rate.values,
            'Population count': bin_sizes.values,
            'Population %': pop_pct.values,
            'Good_count': good_count.values,
            'Bad_count': bad_count.values,
            'P_Good': p_good.values,
            'P_Bad': p_bad.values,
            'Accum_P_Good': accum_p_good.values,
            'Accum_P_Bad': accum_p_bad.values,
            'WOE': woe.values,
            'IV': iv_per_bin.values,
            'Gini': gini_per_bin.values,
            'JS': js_per_bin.values
        })

        # Formatting
        metrics_df['Bad Rate'] = metrics_df['Bad Rate'].map('{:.2%}'.format)
        metrics_df['Good Rate'] = metrics_df['Good Rate'].map('{:.2%}'.format)
        metrics_df['Population %'] = metrics_df['Population %'].map('{:.2%}'.format)
        metrics_df['P_Good'] = metrics_df['P_Good'].map('{:.2%}'.format)
        metrics_df['P_Bad'] = metrics_df['P_Bad'].map('{:.2%}'.format)
        metrics_df['Accum_P_Good'] = metrics_df['Accum_P_Good'].map('{:.2%}'.format)
        metrics_df['Accum_P_Bad'] = metrics_df['Accum_P_Bad'].map('{:.2%}'.format)
        
        metrics_df['Population count'] = metrics_df['Population count'].map('{:,}'.format)
        metrics_df['Good_count'] = metrics_df['Good_count'].map('{:,}'.format)
        metrics_df['Bad_count'] = metrics_df['Bad_count'].map('{:,}'.format)
        
        metrics_df['WOE'] = metrics_df['WOE'].map('{:.4f}'.format)
        metrics_df['IV'] = metrics_df['IV'].map('{:.4f}'.format)
        metrics_df['Gini'] = metrics_df['Gini'].map('{:.4f}'.format)
        metrics_df['JS'] = metrics_df['JS'].map('{:.4f}'.format)

        # Add totals row
        totals_row = {
            'Bin': 'Total',
            'Bad Rate': '',  # Empty string instead of NaN
            'Good Rate': '',
            'Population count': f"{bin_sizes.sum():,}",
            'Range': '',
            'Population %': '',
            'Good_count': f"{good_count.sum():,}",
            'Bad_count': f"{bad_count.sum():,}",
            'P_Good': '',
            'P_Bad': '',
            'Accum_P_Good': '',
            'Accum_P_Bad': '',
            'WOE': '',
            'IV': f"{iv_total:.4f}",
            'Gini': f"{gini_total:.4f}",
            'JS': f"{js_total:.4f}"
        }

        # table
        totals_df = pd.DataFrame([totals_row])
        metrics_df = pd.concat([metrics_df, totals_df], ignore_index=True)


        # Create plot.
        import plotly.graph_objects as go

        fig = go.Figure()

        # Bar traces
        fig.add_trace(go.Bar(x=labels, y=bad_rate, name='Bad Rate', legendgroup='group1', marker_color='orangered',
                            text=[f"{v:.1%}" for v in bad_rate], textposition='inside', insidetextanchor='end',
                            textangle=-45, yaxis='y1', offsetgroup=1))

        fig.add_trace(go.Bar(x=labels, y=pop_pct, name='Population %', marker_color='cornflowerblue', legendgroup='group2',
                            text=[f"{v:.1%}" for v in pop_pct], textposition='inside', insidetextanchor='end',
                            textangle=-45, yaxis='y1', offsetgroup=2))

        # WOE trace
        fig.add_trace(go.Scatter(x=labels, y=woe, name='WOE', legendgroup='group3', mode='lines+markers',
                                marker_color='green', line=dict(width=3), yaxis='y2'))

        # IV, Gini, JS traces
        fig.add_trace(go.Scatter(x=labels, y=iv_per_bin, name='IV per bin', legendgroup='group4', mode='lines+markers',
                                marker_color='purple', line=dict(width=3, dash='dot'), yaxis='y3'))

        fig.add_trace(go.Scatter(x=labels, y=gini_per_bin, name='Gini per bin', legendgroup='group5', mode='lines+markers',
                                marker_color='orange', line=dict(width=3, dash='dash'), yaxis='y3'))

        fig.add_trace(go.Scatter(x=labels, y=js_per_bin, name='JS per bin', legendgroup='group6', mode='lines+markers',
                                marker_color='brown', line=dict(width=3, dash='longdash'), yaxis='y3'))

        # Layout
        fig.update_layout(
            title=dict(
                text=f'Feature: {feature_name}',
                x=0.6,  # Horizontal position (0 = left, 0.5 = center, 1 = right)
                y=0.96, # Vertical position (0 = bottom, 1 = top)
                xanchor='center',
                yanchor='top',
                font=dict(size=16, family='Arial', color='black')
            ),
            xaxis=dict(
                tickangle=-45,
                automargin=True,
                type='category',
                tickfont=dict(size=9)
            ),

            yaxis=dict(
                title='Bad Rate / Population %',
                side='left',
                showgrid=False,
                tickformat='.0%',
                titlefont=dict(size=10),
                linecolor='black'
            ),
            yaxis2=dict(
                title='WOE',
                overlaying='y',
                side='right',
                showgrid=False,
                tickformat='.2f',
                tickfont=dict(color="green"),
                titlefont=dict(size=10, color="green"),
                linecolor='green'
            ),
            yaxis3=dict(
                title='IV / Gini / JS',
                overlaying='y',
                side='left',
                position=0.99,
                showgrid=False,
                tickformat='.3f',
                tickfont=dict(color="orange"),
                titlefont=dict(size=10, color="orange"),
                linecolor='orange'
            ),
            barmode='group',
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=-0.35,
                xanchor="center",
                x=0.5,
                font=dict(size=9)
            ),
            height=600,
            width=1150,
            margin=dict(l=40, r=100, t=40, b=100),
            hovermode='x unified',
            legend_itemclick='toggle',
            legend_itemdoubleclick='toggleothers',
            updatemenus=[dict(
                type="buttons",
                direction="right",
                x=0.5,
                y=1.15,
                showactive=True,
                buttons=[
                    dict(label="▶ Show All", method="update", args=[{"visible": [True]*6}]),
                    dict(label="Bad Rate Only", method="update", args=[{"visible": [True, False, False, False, False, False]}]),
                    dict(label="Population % Only", method="update", args=[{"visible": [False, True, False, False, False, False]}]),
                    dict(label="WOE Only", method="update", args=[{"visible": [False, False, True, False, False, False]}]),
                    dict(label="IV/Gini/JS Only", method="update", args=[{"visible": [False, False, False, True, True, True]}]),
                ]
            )]
        )

        fig.show()


# #################################################################

        feature_min = f"{int(range_min):,}"
        feature_max = f"{int(range_max):,}"


        print("\n" * 1)
        print(f"feature_min , feature_max: ({range_min:,}) , ({range_max:,})")
        print(f"Total IV: {iv_total:.6f}")
        print(f"Overall Gini (Weighted): {gini_total:.6f}")
        print(f"Total JS: {js_total:.6f}")
        # print()
        
        r = list(np.round(bins,3))

        # Format for display with commas
        formatted_display = "[" + ", ".join(f"{val:,}" for val in r) + "]"
        print(f"{feat}_bins = {formatted_display}")

        # ###################################### When filtering is Enabled, replace -inf/inf with min and max

        if bin_purpose:

            r = list(np.round(bins,3))

            # Replace -inf and inf with actual min/max
            r_modified = []
            for val in r:
                if val == -np.inf:
                    replacement = range_min
                elif val == np.inf:
                    replacement = range_max
                else:
                    replacement = val
                r_modified.append(replacement)  

            # Remove duplicates while preserving order
            seen = set()
            x_unique = []
            for val in r_modified:
                if val not in seen:
                    x_unique.append(val)
                    seen.add(val)

            # Format for display with commas
            formatted_display = "[" + ", ".join(f"{val:,}" for val in x_unique) + "]"
            print(f"{feat}_bins_with_Noise_Filteration = {formatted_display}")

        # #######################################################

        table_html = f'''
        <div style="
            max-height: 600px;
            width:1150px; 
            # overflow: auto;
            font-size: 12px;
            margin: 10px 0;
            border: 1px solid #ddd;
            padding: 5px;
            line-height: 1.1;
        ">
            {metrics_df.to_html(escape=False, index=True)}
        </div>
        '''
        display(HTML(table_html))
        
        return metrics_df

def check_constraints(data_input, bins, bad_rate_threshold, min_pop_pct, max_pop_pct, check_monotonicity=True, check_woe_monotonicity=True):
    try:
        data = data_input.copy()
        current_bins = np.unique(bins)
        if len(current_bins) < 2:
            return {'constraints_met': False, 'error': 'Too few bins', 'bin_count': len(current_bins) - 1}

        data['bin'] = pd.cut(data['feature'], bins=current_bins, include_lowest=True, duplicates='drop', right=False)
        grouped = data.groupby('bin', observed=False)
        bin_totals = grouped.size()

        pop_constraint_met = True
        if bin_totals.sum() > 0:
            bin_counts_norm = bin_totals / bin_totals.sum()
            pop_tolerance = 1e-9
            min_pop_actual = bin_counts_norm.min()
            max_pop_actual = bin_counts_norm.max()
            pop_constraint_met = (min_pop_actual >= min_pop_pct - pop_tolerance) and (max_pop_actual <= max_pop_pct + pop_tolerance)
        else:
            pop_constraint_met = False

        good_sum = grouped['target'].sum()
        bad_sum = bin_totals - good_sum
        bad_rates = (bad_sum / bin_totals).fillna(0).to_numpy()

        total_good = data['target'].sum()
        total_bad = len(data) - total_good
        epsilon = 1e-6
        bin_intervals = data['bin'].cat.categories
        p_good = (good_sum / total_good) if total_good > 0 else pd.Series(np.zeros(len(bin_intervals)), index=bin_intervals)
        p_bad = (bad_sum / total_bad) if total_bad > 0 else pd.Series(np.zeros(len(bin_intervals)), index=bin_intervals)
        p_good = p_good.reindex(bin_intervals).fillna(0)
        p_bad = p_bad.reindex(bin_intervals).fillna(0)

        woe = np.log((p_good + epsilon) / (p_bad + epsilon)).to_numpy()
        iv_components = (p_good - p_bad) * woe
        iv_total = iv_components.sum()

        bad_rate_constraint_met = True
        if len(bad_rates) > 1:
            bad_rate_diffs = np.abs(np.diff(bad_rates))
            bad_rate_constraint_met = np.all(bad_rate_diffs >= bad_rate_threshold - 1e-9)

        # #############################################################

        bad_rate_is_monotonic = True
        if check_monotonicity and len(bad_rates) > 1:
            diffs_br = np.diff(bad_rates)
            br_tolerance = 1e-9
            trend_up = detect_monotonic_trend(bad_rates)
            if trend_up: bad_rate_is_monotonic = np.all(diffs_br >= -br_tolerance)
            else: bad_rate_is_monotonic = np.all(diffs_br <= br_tolerance)

        # bad_rate_is_monotonic = True
        # if check_monotonicity and len(bad_rates) > 1:
        #     bad_rate_is_monotonic = detect_monotonic_trend(bad_rates)
        # else:
        #     bad_rate_is_monotonic = None  # Don't check when unchecked

        # #######################################################

        woe_is_monotonic = True
        if check_woe_monotonicity and len(woe) > 1:
            diffs_woe = np.diff(woe)
            woe_tolerance = 1e-9
            woe_is_monotonic = np.all(diffs_woe >= -woe_tolerance) or np.all(diffs_woe <= woe_tolerance)

        constraints_met = pop_constraint_met and bad_rate_constraint_met and bad_rate_is_monotonic and woe_is_monotonic
        return {
            'constraints_met': constraints_met, 'pop_constraint_met': pop_constraint_met,
            'bad_rate_constraint_met': bad_rate_constraint_met, 'bad_rate_is_monotonic': bad_rate_is_monotonic,
            'woe_is_monotonic': woe_is_monotonic, 'iv': iv_total if np.isfinite(iv_total) else 0,
            'bin_count': len(current_bins) - 1, 'error': None
        }
    except Exception as e:
        return {'constraints_met': False, 'error': str(e), 'bin_count': len(np.unique(bins)) - 1}

def check_constraints_custom(data, bins, bad_rate_threshold, check_population=True, min_pop_pct=0.05, max_pop_pct=0.21, check_monotonicity=True):
    min_pop_to_use = min_pop_pct if check_population else 0.0
    max_pop_to_use = max_pop_pct if check_population else 1.0
    result = check_constraints(
        data_input=data, bins=bins, bad_rate_threshold=bad_rate_threshold,
        min_pop_pct=min_pop_to_use, max_pop_pct=max_pop_to_use,
        check_monotonicity=check_monotonicity, check_woe_monotonicity=True
    )
    if check_population:
        pop_check_orig = check_constraints(data, bins, bad_rate_threshold, min_pop_pct, max_pop_pct, False, False)
        result['pop_constraint_met'] = pop_check_orig['pop_constraint_met']
        result['constraints_met'] = (result['pop_constraint_met'] and
                                     result['bad_rate_constraint_met'] and
                                     result['bad_rate_is_monotonic'] and
                                     result['woe_is_monotonic'])
    return result


def find_optimal_bins(data_input, min_bins=3, max_bins=20, min_pop_pct=0.05, max_pop_pct=0.21, bad_rate_threshold=0.04, output_area=None, apply_pop_constraints=True, apply_monotonicity=True, iv_suspicious_threshold=3, special_val=None):

    regular_data, special_data = filter_special_values(data_input, special_val)

    if regular_data.empty:
        if output_area:
            with output_area:
                print("Error: No regular data available for binning after filtering special values.")
        return None

    results = []
    feature_min = regular_data['feature'].min()
    feature_max = regular_data['feature'].max()
    min_bins = max(2, min_bins)
    max_bins = max(min_bins, max_bins)

    log_lines = []
    def log_message(msg, is_progress=False):
        log_lines.append(msg)
        if output_area:
            with output_area:
                clear_output(wait=True)
                print("\n".join(log_lines))

    log_message("--- Starting Optimal Bin Search ---")
    log_message(f"Target Bins: [{min_bins}, {max_bins}]")
    log_message(f"Constraints: Pop={apply_pop_constraints}, Mono={apply_monotonicity}")

    for n_initial_bins in range(min_bins, max_bins + 1):
        log_message(f"\nTrying {n_initial_bins} initial bins...", is_progress=True)
        try:
            bins = create_initial_bins_with_infinity(regular_data, n_initial_bins)
            if len(bins) < 2:
                log_message(f"  Skipped: Too few unique quantiles.")
                continue
        except Exception as e:
            log_message(f"  Skipped: Error generating initial bins - {e}")
            continue

        current_bins = bins.copy()
        step_failed = False

        if apply_pop_constraints:
            current_bins = enforce_population_constraints(regular_data, current_bins, min_pct=min_pop_pct, max_pct=max_pop_pct)
            if len(current_bins) < 2: step_failed = True
        if not step_failed:
            current_bins = merge_bins_by_bad_rate(regular_data, current_bins, threshold=bad_rate_threshold)
            if len(current_bins) < 2: step_failed = True
        if apply_monotonicity and not step_failed:
            current_bins = enforce_monotonic_bad_rate(regular_data, current_bins)
            if len(current_bins) < 2: step_failed = True
        if step_failed:
            log_message(f"  -> Failed during transformation (bins < 2).")
            continue

        # Rest of the function remains unchanged
        final_check_result = check_constraints_custom(
            data=regular_data, bins=current_bins, bad_rate_threshold=bad_rate_threshold,
            check_population=apply_pop_constraints, min_pop_pct=min_pop_pct, max_pop_pct=max_pop_pct,
            check_monotonicity=apply_monotonicity
        )
        constraints_fully_met = final_check_result.get('constraints_met', False)
        iv_value = final_check_result.get('iv', 0)
        actual_bins = final_check_result.get('bin_count', 0)

        status_symbol = "✓" if constraints_fully_met else "✗"
        log_message(f"  Result: {status_symbol} | Final Bins: {actual_bins}, IV: {iv_value:.4f}")
        results.append({
            'initial_bin_count': n_initial_bins, 'actual_bin_count': actual_bins,
            'bins': current_bins, 'constraints_met': constraints_fully_met,
            'iv': iv_value, 'details': final_check_result
        })

    log_message("\n--- Optimal Bin Selection (Refined) ---")
    valid_results = [r for r in results if r['constraints_met']]

    if not valid_results:
        log_message("(!) No solution satisfies ALL requested constraints.")
        log_message("    Consider relaxing constraints or adjusting parameters.")
        return None

    good_iv_results = [r for r in valid_results if r['iv'] <= iv_suspicious_threshold]
    suspicious_iv_results = [r for r in valid_results if r['iv'] > iv_suspicious_threshold]

    best_result = None
    selection_reason = ""

    if good_iv_results:
        best_result = max(good_iv_results, key=lambda x: x['iv'])
        selection_reason = f"Best valid solution with IV <= {iv_suspicious_threshold}"
    elif suspicious_iv_results:
        best_result = min(suspicious_iv_results, key=lambda x: x['iv'])
        selection_reason = f"(!) Best valid solution, BUT IV > {iv_suspicious_threshold} (Suspiciously High). Selected lowest suspicious IV."

    if best_result:
        log_message(f"Selected: {selection_reason}")
        log_message(f"  Initial Bin Attempt: {best_result['initial_bin_count']}")
        log_message(f"  Final Bins: {best_result['actual_bin_count']}")
        log_message(f"  IV: {best_result['iv']:.4f}")
        details = best_result['details']
        pop_msg = f"Pop: {'✓' if details.get('pop_constraint_met') else '✗'}{'' if apply_pop_constraints else ' (n/a)'}"
        br_diff_msg = f"BR Diff: {'✓' if details.get('bad_rate_constraint_met') else '✗'}{'' if bad_rate_threshold > 0 else ' (n/a)'}"
        br_mono_msg = f"BR Mono: {'✓' if details.get('bad_rate_is_monotonic') else '✗'}{'' if apply_monotonicity else ' (n/a)'}"
        woe_mono_msg = f"WOE Mono: {'✓' if details.get('woe_is_monotonic') else '✗'}"
        log_message(f"  Constraints Met: [{pop_msg}, {br_diff_msg}, {br_mono_msg}, {woe_mono_msg}]")
        return best_result['bins']
    else:
        log_message("(!) Error during selection from valid results.")
        return None
    
    
# ####################################################################

def interactive_binning_side_by_side(new_df, filtered_df, feat):
    if new_df.empty:
        print("Cannot start interactive binning: Input data is empty.")
        return
    


    # 2. Define output widgets
    plot_output = Output(layout=Layout(width='1170px', height='610px', border='1px solid #ccc')) # ####################### This is the main influencial width
    log_output = Output(layout=Layout(width='700px', height='600px', border='1px solid #ccc', overflow='auto', padding='5px'))


    
    # 3. Define widget layouts
    widget_layout = Layout(width='250px', margin='5px 0')
    box_layout = Layout(width='150px', height='25px', margin='5px 0')
    box_layout2 = Layout(width='200px', height='25px', margin='5px 0')
    box_layout3 = Layout(width='270px', height='50px', margin='5px 0')


    from ipywidgets import HTML, VBox, Checkbox

    # 4. Define individual widgets (examples; adjust based on your full code)
    general_purpose_binning_checkbox = Checkbox(value=False, description='Enable', indent=False, layout=box_layout3)
    # label = HTML(value='General-Purpose Noise Filter <br>        (Not recommended for Credit Score)')
    label = HTML(value= '<br> General-Purpose Noise Filter')

    bin_count_slider = IntSlider(min=2, max=20, step=1, value=6, description='Manual Bins:', layout=widget_layout, style={'description_width': 'initial', 'handle_color': 'cornflowerblue'})
    find_optimal_checkbox = Checkbox(value=False, description='Find Optimal Bins', indent=False, layout=widget_layout)
    min_bins_text = BoundedIntText(value=2, min=2, max=10, step=1, description='Min Bins (Search):', layout=box_layout, style={'description_width': 'initial'}, disabled=True)
    max_bins_text = BoundedIntText(value=20, min=15, max=30, step=1, description='Max Bins (Search):', layout=box_layout, style={'description_width': 'initial'}, disabled=True)
    pop_constraint_checkbox = Checkbox(value=False, description='Enforce Pop. Constraint', indent=False, layout=widget_layout, style={'description_width': 'initial'})
    min_pop_slider = FloatSlider(value=0.05, min=0.01, max=0.4, step=0.01, description='Min Pop %:', layout=widget_layout, readout_format='.1%', style={'description_width': 'initial', 'handle_color': 'cornflowerblue'})
    max_pop_slider = FloatSlider(value=0.21, min=0.05, max=1.00, step=0.01, description='Max Pop %:', layout=widget_layout, readout_format='.1%', style={'description_width': 'initial', 'handle_color': 'cornflowerblue'})
    merge_checkbox = Checkbox(value=True, description='Merge by Bad Rate', indent=False, layout=box_layout)
    threshold_slider = FloatSlider(value=0.0, min=0.0, max=0.05, step=0.001, description='Merge Threshold:', layout=widget_layout, readout_format='.2%', style={'description_width': 'initial', 'handle_color': 'cornflowerblue'}, disabled=False)
    monotonic_checkbox = Checkbox(value=False, description='Enforce Monotonic BR', indent=False, layout=box_layout)
    iv_threshold_slider = FloatSlider(value=3, min=0.0, max=3.0, step=0.01, description='Suspicious IV >', layout=widget_layout, readout_format='.2f', style={'description_width': 'initial', 'handle_color': 'red'}, disabled=True)
    special_val_text = Text(value='', description='Special Value:', layout=box_layout2, style={'description_width': '100px'})  

    widgets_dict = {'bin_purpose': general_purpose_binning_checkbox,
        'new_bin_count': bin_count_slider, 'pop_constraint': pop_constraint_checkbox,
        'merge': merge_checkbox, 'threshold': threshold_slider, 'monotonic': monotonic_checkbox,
        'min_pop': min_pop_slider, 'max_pop': max_pop_slider, 'find_optimal': find_optimal_checkbox,
        'min_bins': min_bins_text, 'max_bins': max_bins_text, 'iv_threshold': iv_threshold_slider,
        'special_val': special_val_text 
    }

    # ###################################################################

    # def toggle_optimal_widgets(change):
    #     is_optimal = change['new']
    #     bin_count_slider.disabled = is_optimal
    #     min_bins_text.disabled = not is_optimal
    #     max_bins_text.disabled = not is_optimal
    #     iv_threshold_slider.disabled = not is_optimal
    # find_optimal_checkbox.observe(toggle_optimal_widgets, names='value')

    # def toggle_pop_widgets(change):
    #     pop_active = change['new']
    #     min_pop_slider.disabled = not pop_active
    #     max_pop_slider.disabled = not pop_active
    # pop_constraint_checkbox.observe(toggle_pop_widgets, names='value')

    # def toggle_merge_widgets(change):
    #     merge_active = change['new']
    #     threshold_slider.disabled = not merge_active
    # merge_checkbox.observe(toggle_merge_widgets, names='value')



    def toggle_pop_widgets(change):
        pop_active = change['new']
        min_pop_slider.disabled = not pop_active
        max_pop_slider.disabled = not pop_active
        if not pop_active:
            min_pop_slider.value = 0.05  # Reset to default
            max_pop_slider.value = 0.21  # Reset to default
    pop_constraint_checkbox.observe(toggle_pop_widgets, names='value')


    def toggle_merge_widgets(change):
        merge_active = change['new']
        threshold_slider.disabled = not merge_active
        if not merge_active:
            threshold_slider.value = 0.0  # Reset to default
    merge_checkbox.observe(toggle_merge_widgets, names='value')
    
   
    
    def toggle_optimal_widgets(change):
        is_optimal = change['new']
        bin_count_slider.disabled = is_optimal
        pop_constraint_checkbox.disabled = not is_optimal   # is dimmed when optimal is not checked
        min_bins_text.disabled = not is_optimal
        max_bins_text.disabled = not is_optimal
        iv_threshold_slider.disabled = not is_optimal

        # Control description color based on state
        if not is_optimal:
            bin_count_slider.value = 6      # Reset to default
            pop_constraint_checkbox.value = False  # Uncheck when optimal is off
        else:
            min_bins_text.value = 2         # Reset to default
            max_bins_text.value = 20        # Reset to default
            iv_threshold_slider.value = 3   # Reset to default
    find_optimal_checkbox.observe(toggle_optimal_widgets, names='value')

    # #############################################################################

    toggle_optimal_widgets({'new': find_optimal_checkbox.value})
    toggle_pop_widgets({'new': pop_constraint_checkbox.value})
    toggle_merge_widgets({'new': merge_checkbox.value})


    def update_binning(bin_purpose, new_bin_count, pop_constraint, merge, threshold, monotonic, min_pop, max_pop, find_optimal, min_bins, max_bins, iv_threshold, special_val):
        log_output.clear_output(wait=True)
        plot_output.clear_output(wait=True)
        final_bins = None


        # Parse special_val
        special_val_parsed = None
        if special_val.strip().lower() == 'nan':
            special_val_parsed = np.nan
        elif special_val.strip():
            try:
                special_val_parsed = float(special_val)
            except ValueError:
                pass

        # general_purpose_binning
        data = filtered_df.copy() if bin_purpose else new_df.copy()

        # Split data into regular and special
        if special_val_parsed is not None:
            if np.isnan(special_val_parsed):
                mask = data['feature'].isna()
            else:
                mask = data['feature'] == special_val_parsed
            special_data = data[mask]
            regular_data = data[~mask]
        else:
            regular_data = data.copy()
            special_data = pd.DataFrame(columns=data.columns)

        with log_output:
            print("--- Starting Update ---")
            print(f"Mode: {'Optimal' if find_optimal else 'Manual'}")
            print(f"general_purpose_binning: {bin_purpose}")
            print(f"Special Value: {special_val_parsed if special_val_parsed is not None else 'None'}")

            if find_optimal:
                print(f"Searching bins [{min_bins}-{max_bins}], Suspicious IV > {iv_threshold:.2f}")
                final_bins = find_optimal_bins(
                    data_input=regular_data, min_bins=min_bins, max_bins=max_bins,
                    min_pop_pct=min_pop, max_pop_pct=max_pop, bad_rate_threshold=threshold,
                    output_area=log_output, apply_pop_constraints=pop_constraint, apply_monotonicity=monotonic,
                    iv_suspicious_threshold=iv_threshold
                )
                if final_bins is None or len(final_bins) < 2:
                    # print("Optimal binning failed or returned insufficient bins.")
                    print("\n--- Optimal Binning Failed ---")
                    print("No solution found meeting all constraints.")
                    print("Try adjusting parameters (e.g., relax constraints, change bin range).")
                    with plot_output:
                        print("Optimal binning failed. No plot to display.")
                    print("\n--- Update Complete ---")
                    return
                                    
            else:
                print(f"\nManual Mode: Starting with {new_bin_count} bins.")
                print(f"general_purpose_binning: {bin_purpose}")
                try:

                    feature_min = regular_data['feature'].min()
                    feature_max = regular_data['feature'].max()

                    # Always start bins from -inf, special value or not
                    bins = create_initial_bins_with_infinity(regular_data, new_bin_count)
                    if len(bins) < 2:
                        raise ValueError("Too few unique quantiles")
                    current_bins = bins.copy()
                    print("Initial bins:", np.round(current_bins, 3))

                    if pop_constraint:
                        print("Enforcing population...")
                        current_bins = enforce_population_constraints(regular_data, current_bins, min_pct=min_pop, max_pct=max_pop)
                        print("After Pop:", np.round(current_bins, 3))
                        if len(current_bins) < 2:
                            raise ValueError("Population enforcement failed")
                        
                        
                    if merge:
                        print("Merging...")
                        current_bins = merge_bins_by_bad_rate(regular_data, current_bins, threshold)
                        print("After Merge:", np.round(current_bins, 3))
                        if len(current_bins) < 2:
                            raise ValueError("Merge failed")
                        
                    if monotonic:
                        print("Enforcing monotonicity...")
                        current_bins = enforce_monotonic_bad_rate(regular_data, current_bins)
                        print("After Monotonicity:", np.round(current_bins, 3))
                        if len(current_bins) < 2:
                            raise ValueError("Monotonicity enforcement failed")
                    final_bins = current_bins
                except ValueError as e:
                    print(f"Manual Error: {e}")
                    final_bins = None
                except Exception as e:
                    print(f"Unexpected Manual Error: {e}")
                    final_bins = None

            if final_bins is not None and len(final_bins) >= 2:
                print("\n--- Assigning Bins ---")
                
                # Assign regular data to bins
                regular_data.loc[:, 'bin'] = pd.cut(
                    regular_data['feature'], 
                    bins=final_bins, 
                    include_lowest=True, 
                    right=False
                )
                
                # Handle special values only if they exist
                if not special_data.empty:
                    special_data.loc[:, 'bin'] = 'special'
                    combined_data = pd.concat([regular_data, special_data], ignore_index=True)
                else:
                    combined_data = regular_data

                print("\n--- Calculating Final Metrics ---")
                # Calculate metrics (unchanged)
                (bad_rate, woe, bin_sizes, bin_range, range_min, range_max, iv_per_bin, iv_total,
                gini_per_bin, gini_total, js_per_bin, js_total, p_bad,
                good_count, bad_count, p_good, good_rate, accum_p_good, accum_p_bad) = calculate_metrics(combined_data, final_bins)

                # Plot binning
                plot_binning(bad_rate, woe, bin_sizes, bin_range, range_min, range_max, iv_per_bin, iv_total,
                            gini_per_bin, gini_total, js_per_bin, js_total,
                            final_bins, p_bad, feat, plot_output,
                            good_count, bad_count, p_good, good_rate, accum_p_good, accum_p_bad, bin_purpose)


                print("\n--- Final Constraints Check ---")
                final_constraints = check_constraints_custom(
                    combined_data, final_bins, threshold,
                    check_population=pop_constraint, min_pop_pct=min_pop, max_pop_pct=max_pop,
                    check_monotonicity=monotonic
                )

                general_purpose_msg = f"Gen_purpose: {'✓' if final_constraints.get('bad_rate_constraint_met') else '✗'}{'' if merge else ' (Not Checked)'}"


                pop_msg = f"Pop: {'✓' if final_constraints.get('pop_constraint_met') else '✗'}{'' if pop_constraint else ' (Not Checked)'}"

                br_diff_msg = f"BR Diff: {'✓' if final_constraints.get('bad_rate_constraint_met') else '✗'}{'' if merge else ' (Not Checked)'}"

                br_mono_msg = f"BR Mono: {'✓' if final_constraints.get('bad_rate_is_monotonic') else '✗'}{'' if monotonic else ' (Not Checked)'}"
                # br_mono_result = final_constraints.get('bad_rate_is_monotonic')
                # if monotonic: br_mono_msg = f"BR Mono: {'✓' if br_mono_result else '✗'}"
                # else: br_mono_msg = f"BR Mono: — (Not Checked)"

                woe_mono_msg = f"WOE Mono: {'✓' if final_constraints.get('woe_is_monotonic') else '✗'}"
                
                print(f"Status: [{pop_msg}, {br_diff_msg}, {br_mono_msg}, {woe_mono_msg}]")
                print("\nNote:")
                print(" * ✓or✗ = The bins satisfy/dissatisfy the constraint. And (-) means the bins are not being checked")
                print(" * Not Checked = The constraint was not enforced during binning")
            else:
                print("\n--- Binning Failed ---")
                with plot_output:
                    print("Binning failed. No plot to display.")
            print("\n--- Update Complete ---")

    interactive_controller = interactive(update_binning, **widgets_dict)

    # 5. Group widgets into boxes
    # Layout the UI
    business_box = VBox([label, general_purpose_binning_checkbox])
    manual_box = VBox([bin_count_slider])
    optimal_box = VBox([find_optimal_checkbox, min_bins_text, max_bins_text])
    constraint_box = VBox([pop_constraint_checkbox, min_pop_slider, max_pop_slider,
                           merge_checkbox, threshold_slider, monotonic_checkbox, iv_threshold_slider, special_val_text])

    # 6. Define control_pane
    control_pane = VBox([business_box, manual_box, optimal_box, constraint_box],
                        layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))
    
        
    # 7. Define right_side
    right_side = VBox([control_pane, log_output], layout=Layout(width='1200px', margin='0 0 0 1px'))
    main_layout = HBox([plot_output, right_side], layout=Layout(width='1900px'), height='1000px') # ###################Overall width of the layout


# ###################################################################################

    display(main_layout)
    for name, widget in widgets_dict.items():
        widget.observe(
            lambda change, bound_widget=interactive_controller, key_name=name: setattr(bound_widget, key_name, change['new']),
            names='value'
        )
    initial_kwargs = {name: widget.value for name, widget in widgets_dict.items()}
    update_binning(**initial_kwargs)

def run_interactive_binning(new_df, filtered_df, feat):
    if new_df is None or not isinstance(new_df, pd.DataFrame) or new_df.empty:
        print("Error: Input data is missing, not a DataFrame, or empty.")
        return
    if feat not in df.columns:
        print(f"Error: Feature '{feat}' not found in DataFrame columns: {df.columns.tolist()}")
        return
    if 'target' not in new_df.columns:
        print(f"Error: Required 'target' column not found in DataFrame columns: {new_df.columns.tolist()}")
        return
    clear_output(wait=True)

    # In def below, data = filtered_df.copy() if bin_purpose else new_df.copy()
    interactive_binning_side_by_side(new_df, filtered_df, feat)

# Run the tool
print("Original Data shape:", new_df.shape)
print("Data without outliers shape:", filtered_df.shape)
print("Feature:", feat)
run_interactive_binning(new_df, filtered_df, feat)

### *Using the list to do the split manually*

In [6]:
special_value = -99999
# globals()[f'{feat}_bins'] = [-np.inf, 118, 1,000, 2,000, 2,818, 4,000, 5,000, 9,600, 17,344, np.inf]
globals()[f'{feat}_bins'] = [-np.inf, 30.9, 47.3, 59.7, 70.8, 82.7, np.inf]

In [7]:

def calculate_woe(data, bin_column, target_column, positive = positive, negative = negative, positive_label=positive_label, negative_label=negative_label, epsilon=1e-10, mp=1):
    """
    Calculate Weight of Evidence with configurable positive/negative labels
    Returns variables named according to the actual label values
    """
    grouped = data.groupby(bin_column, observed=False)

    # Count cases for each label (dynamically named)
    label_counts = {}
    label_counts[f'{positive}_counts'] = grouped.apply(lambda x: (x[target_column] == positive_label).sum())
    total_in_bin = grouped.size()
    label_counts[f'{negative}_counts'] = total_in_bin - label_counts[f'{positive}_counts']

    # Total counts
    total_counts = {}
    total_counts[f'total_{positive}'] = (data[target_column] == positive_label).sum()
    total_counts[f'total_{negative}'] = (data[target_column] == negative_label).sum()

    # Calculate percentages
    per_positive = label_counts[f'{positive}_counts'] / (total_counts[f'total_{positive}'] + epsilon) * 100
    per_negative = label_counts[f'{negative}_counts'] / (total_counts[f'total_{negative}'] + epsilon) * 100

    ratio = (per_positive + epsilon) / (per_negative + epsilon)

    woe = np.where((label_counts[f'{positive}_counts'] == 0) | (label_counts[f'{negative}_counts'] == 0), 0, np.log(ratio))
    woe = mp * np.where(np.isinf(woe), 0, woe)
    woe_series = pd.Series(woe, index=label_counts[f'{positive}_counts'].index, name=bin_column)
    
    return (label_counts[f'{positive}_counts'], 
            label_counts[f'{negative}_counts'], 
            total_counts[f'total_{positive}'], 
            total_counts[f'total_{negative}'], 
            woe_series)

In [8]:
# Keep a list for bin_number & bin_range
bin_lnk_num = None
feat_count = 0

# Retrieve binning splits
binning_splits = globals()[f'{n}_bins']
binning_splits = set(binning_splits)

# Add special value or fallback dummy value
try:
    binning_splits.add(special_value + 1)
except NameError:
    binning_splits.add(-1e+15)

# Convert back to a sorted list
binning_splits = sorted(binning_splits)

# Print binning splits with scientific notation formatting
formatted_splits = [f"{x:.0e}" if isinstance(x, float) and abs(x) >= 1e+6 else str(x) for x in binning_splits]

# Convert back to a sorted list
binning_splits = sorted(binning_splits)

# ✅ Print binning_splits as numeric values
print()
print(f'{feat_count} - Feature = {n}')

# #####################################################
# Formating (-1e+15)
def format_for_display(val):
    if val in [np.inf, -np.inf]:
        return str(val)
    elif abs(val) >= 1e+6 or abs(val) < 1e-2:
        return f"{val:.0e}"
    else:
        return f"{val:.1f}"

formatted_display = "[" + ", ".join(format_for_display(x) for x in binning_splits) + "]"
print(f'binning_splits = {formatted_display}')
# ########################################################
print()

# Apply binning
new_df = good_has_ones(df, target_col, bad=bad)
ranking = new_df[[n, target_col]].copy()
ranking[f'{n}_bins'] = pd.cut(ranking[n], bins=binning_splits, right=False, include_lowest=False)
ranking = pd.DataFrame(ranking)

# Encode bins with OrdinalEncoder
encoder = ce.OrdinalEncoder(cols=[f'{n}_bins'], return_df=True)
ranking_encoded = encoder.fit_transform(ranking)

# Get unique bins sorted by left edge
unique_bins = ranking[f'{n}_bins'].dropna().unique()
sorted_bins = sorted(unique_bins, key=lambda x: x.left)

# Create mapping dictionary
mapping_dict = {}
if sorted_bins:
    if sorted_bins[0].left == -np.inf:
        ranking_encoded[f'{n}_bins'] = ranking_encoded[f'{n}_bins'] - 1
        mapping_dict = {bin_: i for i, bin_ in enumerate(sorted_bins)}
    else:
        mapping_dict = {bin_: i + 1 for i, bin_ in enumerate(sorted_bins)}

# ✅ Format bin labels with scientific notation for large/small values
def format_edge(val):
    if val in [np.inf, -np.inf]:
        return str(val)
    elif abs(val) >= 1e+6 or abs(val) < 1e-2:
        return f"{val:.0e}"
    else:
        return f"{val:.1f}"

def format_bin(interval):
    left = format_edge(interval.left)
    right = format_edge(interval.right)
    return f"[{left}, {right})"

# Print the mapping details
print(f"Mapping for {n}")
for bin_, num in mapping_dict.items():
    print(f"{format_bin(bin_)} → {num}")
print()




# #################################################################################
# ########################### The below for special bin to take the highest number

# bin_first_with_special_val = False
# # Convert to categorical with sorted bins for consistent encoding
# unique_bins = ranking[f'{n}_bins'].dropna().unique()
# sorted_bins = sorted(unique_bins, key=lambda x: x.left)

# ranking[f'{n}_bins'] = pd.Categorical(ranking[f'{n}_bins'], categories=sorted_bins, ordered=True)

# # Encode bins with OrdinalEncoder (starts at 1)
# encoder = ce.OrdinalEncoder(cols=[f'{n}_bins'], return_df=True)
# ranking_encoded = encoder.fit_transform(ranking)

# # Get total number of bins
# total_bins = len(sorted_bins)

# # Create mapping dictionary for printing
# mapping_dict = {}

# # Adjust bin numbers if first bin starts with -inf
# if sorted_bins and sorted_bins[0].left == -np.inf:
#     # Create remap dictionary for encoded values
#     remap_dict = {1: total_bins}  # Map first bin to highest number
#     # Map other bins to 1, 2, 3, ..., total_bins-1
#     for i in range(2, total_bins + 1):
#         remap_dict[i] = i - 1

#     # Apply remapping
#     ranking_encoded[f'{n}_bins'] = ranking_encoded[f'{n}_bins'].map(remap_dict)

#     # Create mapping dictionary for printing
#     mapping_dict[sorted_bins[0]] = total_bins
#     for i, bin_ in enumerate(sorted_bins[1:], start=1):
#         mapping_dict[bin_] = i
# else:
#     # No adjustment needed
#     mapping_dict = {bin_: i+1 for i, bin_ in enumerate(sorted_bins)}

# # Print the mapping details
# print(f"Mapping for {n}")
# for bin_, num in mapping_dict.items():
#     print(f"{bin_} → {num}")
# print()


# ##############################################
# ################ Calculate woe for each bin

# woe = calculate_woe(ranking, f'{n}_bins', target_col)
woe = calculate_woe(ranking, f'{n}_bins', target_col, positive = positive, negative = negative, positive_label=positive_label, negative_label=negative_label, epsilon=1e-10, mp=1)

# print()
# woe


0 - Feature = total_rev_hi_lim
binning_splits = [-inf, -99998.0, 30.9, 47.3, 59.7, 70.8, 82.7, inf]

1s are greater than 0s, where bad has 0
Mapping for total_rev_hi_lim
[-inf, -99998.0) → 0
[-99998.0, 30.9) → 1
[82.7, inf) → 2



In [9]:
def bin_statistics(feature_name, good_counts, bad_counts, total_good, total_bad, woe):
    tot = good_counts + bad_counts
    tot_sum = tot.sum()
    
    pct_tot = tot / tot_sum * 100
    pct_good_tot = good_counts / tot_sum * 100
    pct_bad_tot = bad_counts / tot_sum * 100
    good_rate = good_counts / tot * 100
    bad_rate = bad_counts / tot * 100
    pct_good = good_counts / total_good * 100
    pct_bad = bad_counts / total_bad * 100
    iv = (pct_good/100 - pct_bad/100) * woe
    iv_total = iv.sum()
    
    df_bins = pd.DataFrame({
        "Factors": [feature_name] * len(good_counts),
        "all_bins": len(good_counts),

        # ######################################### Replace bin_0 with Special
        # "Bin_num": range(len(good_counts)),
        "Bin_num": ['Special' if i == 0 else i for i in range(len(good_counts))],
        # ########################################

        "tot": tot,
        "%tot": pct_tot.round(4),
        "good": good_counts,
        "bad": bad_counts,
        "%good/tot": pct_good_tot.round(4),
        "%bad/tot": pct_bad_tot.round(4),
        "good_rate": good_rate.round(4),
        "bad_rate": bad_rate.round(4),
        "%good": pct_good.round(4),
        "%bad": pct_bad.round(4),
        "WOE": woe.round(4),
        "IV": iv.round(4)
    })
    
    return df_bins, iv_total

# #######################################

# Calculating WOE for each bin
good_counts, bad_counts, total_good, total_bad, woe_series = calculate_woe(ranking, f'{n}_bins', target_col)

In [10]:
# ################################# Relabeling bins for special value bin

def relabel_bins_dynamic(index):
    new_index = []
    for i, b in enumerate(index):
        if isinstance(b, pd.Interval):
            if i == 0:
                new_index.append('Special')
            elif i == 1:
                # Replace left edge with -inf
                new_index.append(pd.Interval(left=-np.inf, right=b.right, closed=b.closed))
            else:
                new_index.append(b)
        else:
            new_index.append(b)
    return new_index

# ########################################
# Relabeling the bins
woe_series.index = relabel_bins_dynamic(woe_series.index)
good_counts.index = relabel_bins_dynamic(good_counts.index)
bad_counts.index = relabel_bins_dynamic(bad_counts.index)

In [11]:
df_bins, total_iv = bin_statistics(f"{n}_bins", good_counts, bad_counts, total_good, total_bad, woe_series)

# print("\nTotal IV:", round(total_iv, 4))
# df_bins

In [12]:
# Plot the new adjusted bins

df_factor = df_bins.copy()

# Format percentage values for display
df_factor['%tot_text'] = df_factor['%tot'].apply(lambda x: f'{x:.1f}%')
df_factor[f'%{positive}/tot_text'] = df_factor[f'%{positive}/tot'].apply(lambda x: f'{x:.1f}%')
df_factor[f'%{negative}/tot_text'] = df_factor[f'%{negative}/tot'].apply(lambda x: f'{x:.0f}%')

# Create figure with subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])

# %tot bars (light blue) - assigned to y3
fig.add_trace(
    go.Bar(
        x=df_factor['Bin_num'],
        y=df_factor['%tot'],
        name='%tot',
        text=df_factor['%tot_text'],
        textposition='inside',
        textangle= -90,  # Vertical text from bottom to top
        insidetextanchor='middle',
        textfont=dict(color='black', size=12, family='Arial'),
        marker_color='lightblue',
        opacity=0.7,
        width=0.3,
        offset=-0.15,
        yaxis='y3'
    )
)

# %negative/tot bars (red) - assigned to y3
fig.add_trace(
    go.Bar(
        x=df_factor['Bin_num'],
        y=df_factor[f'%{negative}/tot'],
        name=f'%{negative}/tot',
        text=df_factor[f'%{negative}/tot_text'],
        textposition='inside',
        # textangle= -90,  # Vertical text from bottom to top
        insidetextanchor='start',
        textfont=dict(color='white', size=10),
        marker_color='red',
        opacity=0.7,
        width=0.3,
        offset=0.15,
        yaxis='y3'
    )
)

# %positive/tot bars (blue) - stacked on %negative/tot, assigned to y3
fig.add_trace(
    go.Bar(
        x=df_factor['Bin_num'],
        y=df_factor[f'%{positive}/tot'],
        name=f'%{positive}/tot',
        text=df_factor[f'%{positive}/tot_text'],
        textposition='inside',
        textangle= -90,  # Vertical text from bottom to top
        insidetextanchor='middle',
        textfont=dict(color='white', size=10),
        marker_color='blue',
        opacity=0.7,
        width=0.3,
        offset=0.15,
        base=df_factor[f'%{negative}/tot'],
        yaxis='y3'
    )
)

# Negative_rate line (red) - primary y-axis
fig.add_trace(
    go.Scatter(
        x=df_factor['Bin_num'],
        y=df_factor[f'{negative}_rate'],
        name=f'{negative}_rate',
        mode='lines+markers',
        line=dict(color='red', width=2.5),
        marker=dict(size=8, symbol='diamond'),
        yaxis='y'
    )
)

# WOE line (green) - y2 axis
fig.add_trace(
    go.Scatter(
        x=df_factor['Bin_num'],
        y=df_factor['WOE'],
        name='WOE',
        mode='lines+markers',
        line=dict(color='green', width=2.5, dash='dot'),
        marker=dict(size=8, symbol='x'),
        yaxis='y2'
    )
)

# Update layout with adjustments
# Solution: Adjust layout to avoid overlapping y2, y3 axis titles and plot title

fig.update_layout(
    title=dict(
        text=f'Factor Analysis: {n}',
        x=0.5,
        xanchor='center',
        y=0.95
    ),
    xaxis_title='Bins',
    legend_title='Metrics',
    hovermode='x unified',
    template='plotly_white',
    bargap=0.15,
    margin=dict(r=180),  # Increased top and right margins

# ################################################# Replace bin_0 with Special
    # xaxis=dict(
    #     domain=[0, 0.95],  # Reduce domain to make space for y2/y3 and avoid overlap with legend
    #     tick0=df_factor['Bin_num'].min(),  # Start ticks at the minimum bin number
    #     dtick=1,  # Set tick interval to 1 (integer values only)
    #     tickmode='linear'  # Use linear tick mode for consistent spacing
    # ),

    xaxis=dict(
    domain=[0, 0.95],
    title='Bins',
    type='category',
    categoryorder='array',
    categoryarray=df_factor['Bin_num'].tolist()
    ),
    # ###########################################3

    yaxis=dict(
        title=f"<b>{negative}_rate</b>",
        side='left',
        title_font=dict(size=12, color="red"),
        tickfont=dict(color="red"),
        linecolor='red',                   # Red axis line
        gridcolor='rgba(255, 0, 0, 0.1)',   # Light red grid lines
        range=[0, df_factor[f'{negative}_rate'].max() * 1.3]
    ),

    yaxis2=dict(
        title="<b>WOE</b>",
        anchor='free',
        overlaying='y',
        side='right',
        position= 0.01, 
        title_standoff=10, 
        title_font=dict(size=12, color="green"),
        tickfont=dict(color="green"),
        linecolor='green',
        range=[df_factor['WOE'].min() * 1.1, df_factor['WOE'].max() * 1.1]
    ),
    yaxis3=dict(
        title="<b>Bar Values (%)</b>",
        anchor='x',
        overlaying='y',
        side='right',
        position=0.93,
        title_standoff=30, # title position 70
        title_font=dict(size=12, color="blue"),
        tickfont=dict(color="blue"),
        linecolor='blue',
        range=[0, df_factor['%tot'].max() * 1.2]
    )
)

fig.show()

In [13]:
# Binning Validation

def validate_binning(df, bin_col='Bin_num', woe_col='WOE', iv_col='IV', pct_col='%tot', target_col='bad_rate'):
    result = df.copy()

    # 1. Bin Balance Check
    result['flag_bin_balance'] = np.where(result[pct_col] < 1, 'Low', 'OK')

    # 2. WOE Jump Check
    result['woe_jump'] = result[woe_col].diff().abs().round(4)
    result['flag_woe'] = np.where(result['woe_jump'] > 0.5, 'Jump', 'OK')

    # 3. IV Strength Check
    result['flag_iv'] = np.where(result[iv_col] < 0.01, 'Weak', 'OK')

    # 4. Spearman Monotonicity
    try:
        spearman_corr, _ = spearmanr(result[bin_col], result[target_col])
    except Exception:
        spearman_corr = np.nan

    # 5. Total IV
    total_iv = result[iv_col].sum().round(4)

    # 6. Styling for binning flags
    styled = result.style.applymap(
        lambda x: 'background-color: #fdd' if x == 'Low' else '', subset=['flag_bin_balance']
    ).applymap(
        lambda x: 'background-color: #fdc' if x == 'Jump' else '', subset=['flag_woe']
    ).applymap(
        lambda x: 'background-color: #eef' if x == 'Weak' else '', subset=['flag_iv']
    )

    # 7. Color Legend as Styled DataFrame
    legend_data = {
        'Color': ['🔴 Light Red', '🟠 Light Orange', '🔵 Light Blue'],
        'Column': ['flag_bin_balance', 'flag_woe', 'flag_iv'],
        'Meaning': [
            'Bin has low volume (<1% of total)',
            'WOE jump > 0.5 — may indicate erratic bin behavior or instability between adjacent bins',
            'IV < 0.01 — weak predictive power, consider dropping or merging'
        ]
    }
    legend_df = pd.DataFrame(legend_data)

    legend_styled = legend_df.style.set_table_styles([
        {'selector': 'td', 'props': [('white-space', 'normal'), ('max-width', '400px'), ('text-align', 'left')]},
        {'selector': 'th', 'props': [('text-align', 'left')]}
    ])

    # 8. Summary
    summary = {
        'Total IV': total_iv,
        'Spearman Monotonicity': round(spearman_corr, 4)
    }

    return styled, legend_styled, summary


In [14]:
styled, legend_styled, summary = validate_binning(df_bins)

print(summary)       # Summary stats
display(styled)   # Styled binning table
display(legend_styled)   # Legend table


{'Total IV': 0.0036, 'Spearman Monotonicity': nan}


,Factors,all_bins,Bin_num,tot,%tot,good,bad,%good/tot,%bad/tot,good_rate,bad_rate,%good,%bad,WOE,IV,flag_bin_balance,woe_jump,flag_woe,flag_iv
Special,total_rev_hi_lim_bins,7,Special,56821,38.463000,47974,8847,32.474300,5.988700,84.430100,15.569900,38.036900,40.950700,-0.073800,0.002200,OK,nan,OK,Weak
"[-inf, 30.9)",total_rev_hi_lim_bins,7,1,37,0.025000,33,4,0.022300,0.002700,89.189200,10.810800,0.026200,0.018500,0.345800,0.000000,Low,0.419600,OK,Weak
"[30.9, 47.3)",total_rev_hi_lim_bins,7,2,0,0.000000,0,0,0.000000,0.000000,nan,nan,0.000000,0.000000,0.000000,0.000000,Low,0.345800,OK,Weak
"[47.3, 59.7)",total_rev_hi_lim_bins,7,3,0,0.000000,0,0,0.000000,0.000000,nan,nan,0.000000,0.000000,0.000000,0.000000,Low,0.000000,OK,Weak
"[59.7, 70.8)",total_rev_hi_lim_bins,7,4,0,0.000000,0,0,0.000000,0.000000,nan,nan,0.000000,0.000000,0.000000,0.000000,Low,0.000000,OK,Weak
"[70.8, 82.7)",total_rev_hi_lim_bins,7,5,0,0.000000,0,0,0.000000,0.000000,nan,nan,0.000000,0.000000,0.000000,0.000000,Low,0.000000,OK,Weak
"[82.7, inf)",total_rev_hi_lim_bins,7,6,90871,61.512000,78118,12753,52.879300,8.632700,85.965800,14.034200,61.937000,59.030700,0.048100,0.001400,OK,0.048100,OK,Weak


,Color,Column,Meaning
0,🔴 Light Red,flag_bin_balance,Bin has low volume (<1% of total)
1,🟠 Light Orange,flag_woe,WOE jump > 0.5 — may indicate erratic bin behavior or instability between adjacent bins
2,🔵 Light Blue,flag_iv,"IV < 0.01 — weak predictive power, consider dropping or merging"


## *Feature Characteristics*

In [15]:

# df_report = df_all[[feat, target_col]].copy()

# profile = ProfileReport(df_report, title="Profiling Report", explorative=True)     # ydata_profiling
# # profile.to_file("report.html")

# # For jupyter notebook
# profile.to_widgets()           # Interactive widget view
# profile.to_notebook_iframe()  # Embedded HTML view

In [16]:
# how to save the file in python file
# jupyter nbconvert --to python your_file.ipynb